In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from models import HyperSphericalVAE

In [3]:
from configs import beta_vae
config = beta_vae.get_config()
config.latent_dim = 3
config.batch_size = 16384
config.kld_weight = 1
config

aug_p: 0.0
aug_std: 0.0
batch_size: 16384
dataset: Organoid
device: cuda
epochs: 10000
hidden_dims: !!python/tuple
- 32
- 32
- 32
in_features: 41
kld_weight: 1.0
latent_dim: 3
learning_rate: 0.05
loss_type: beta
max_grad_norm: .inf
model: VAE
output_dir: ./logs/VanillaVAE/
save_loss_every_n_epochs: 10
seed: 12345
weight_decay: 0.0

In [4]:
model = HyperSphericalVAE(config).to(config.device)

In [5]:
model

HyperSphericalVAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=41, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
  )
  (fc_mu): Linear(in_features=32, out_features=3, bias=True)
  (fc_var): Linear(in_features=32, out_features=1, bias=True)
  (decoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affi

In [6]:
from datasets import OrganoidDataset
data = OrganoidDataset()

X_train,y_train = data.train
X_val,y_val = data.val


In [14]:
X_train_batches = torch.split(X_train,split_size_or_sections=config.batch_size)
X_val_batches = torch.split(X_train,split_size_or_sections=config.batch_size)

In [7]:
from torch import optim

optimizer = optim.Adam(model.parameters(),
                       lr=0.005,
                       weight_decay=0.0,
                       )
optimizer.zero_grad()
outputs = model.forward(X_train)
loss = model.loss_function(*outputs, config=config)['loss']
loss.backward()
optimizer.step()

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torch/distributions/distribution.py:45: UserWarning: <class 'models.hyperspherical_vae_extra.distributions.hyperspherical_uniform.HypersphericalUniform'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [22]:
z_mean, z_var = model.encode(X_train_batches[0])

In [19]:
model.encode(X_train_batches[0])[0].shape

torch.Size([16384, 3])

In [20]:
model.encode(X_train_batches[0])[1].shape

torch.Size([16384, 1])

In [21]:
model.reparameterize(*model.encode(X_train_batches[0]))

/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torch/distributions/distribution.py:45: UserWarning: <class 'models.hyperspherical_vae_extra.distributions.hyperspherical_uniform.HypersphericalUniform'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [32]:
from models import VonMisesFisher

VonMisesFisher(z_mean,z_var).sample().shape

torch.Size([16384, 3])

In [6]:
import os
os.system('python3 /home/egor/PycharmProjects/deep_dr/experiment.py --config=configs/hs_vae.py --config.epochs=100')

Global seed set to 12345
  0%|          | 0/100 [00:00<?, ?it/s]

CUDA available:True
Last batch of size 509 is smaller than half batch size 1024 and is dropped


/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torch/distributions/distribution.py:45: UserWarning: <class 'models.hyperspherical_vae_extra.distributions.hyperspherical_uniform.HypersphericalUniform'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
100%|██████████| 100/100 [14:15<00:00,  8.56s/it]
/home/egor/PycharmProjects/deep_dr/venv/lib/python3.10/site-packages/torch/distributions/distribution.py:45: UserWarning: <class 'models.hyperspherical_vae_extra.distributions.hyperspherical_uniform.HypersphericalUniform'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


Memory allocated:202809856
Max memory allocated:209387520
Finished Training in 0:14:15.703343


0